In [ ]:
#part 1
def parse_graph(lines):
    graph = {}
    for line in lines:
        if not line.strip():
            continue
        name, targets = line.split(":")
        name = name.strip()
        targets = targets.strip().split()
        graph[name] = targets
    return graph

def all_paths(graph, start, end):
    paths = []
    stack = [(start, [start])]
    while stack:
        node, path = stack.pop()
        if node == end:
            paths.append(path)
            continue
        if node not in graph:
            continue
        for nxt in graph[node]:
            if nxt not in path:
                stack.append((nxt, path + [nxt]))
    return paths


with open('input.txt','r') as f:
    inputs=[line.strip() for line in f.readlines()]

graph = parse_graph(inputs)
paths = all_paths(graph, "you", "out")

len(paths)


758

In [ ]:
#not working part2
def parse_graph(lines):
    graph = {}
    for line in lines:
        if not line.strip():
            continue
        name, targets = line.split(":")
        name = name.strip()
        targets = targets.strip().split()
        graph[name] = targets
    return graph

def all_paths_with_required(graph, start, end, required_nodes):
    required = set(required_nodes)
    paths = []
    stack = [(start, [start], set([start]) & required)]
    while stack:
        node, path, seen_required = stack.pop()
        if node == end:
            if seen_required == required:
                paths.append(path)
            continue
        if node not in graph:
            continue
        for nxt in graph[node]:
            if nxt not in path:
                new_required = seen_required | (set([nxt]) & required)
                stack.append((nxt, path + [nxt], new_required))
    return paths

with open('input.txt','r') as f:
    inputs=[line.strip() for line in f.readlines()]

graph = parse_graph(inputs)
paths = all_paths_with_required(graph, "svr", "out", ['fft','dac'])

print("Total paths:", len(paths))
for p in paths:
    print(" -> ".join(p))


KeyboardInterrupt: 

In [ ]:
#not working part2
def parse_graph(lines):
    graph = {}
    for line in lines:
        if not line.strip():
            continue
        name, targets = line.split(":")
        name = name.strip()
        targets = targets.strip().split()
        graph[name] = targets
    return graph


def all_paths_with_required_fast(graph, start, end, required_nodes):
    required = set(required_nodes)

    reverse = {node: [] for node in graph}
    for src, outs in graph.items():
        for dst in outs:
            reverse.setdefault(dst, []).append(src)

    reachable_to_end = set([end])
    queue = [end]
    while queue:
        cur = queue.pop()
        for prev in reverse.get(cur, []):
            if prev not in reachable_to_end:
                reachable_to_end.add(prev)
                queue.append(prev)

    paths = []
    stack = [(start, (start,), frozenset([start]) & required)]

    while stack:
        node, path, seen_required = stack.pop()

        if node == end:
            if seen_required == required:
                paths.append(path)
            continue

        if node not in graph:
            continue
        if node not in reachable_to_end:
            continue

        outs = graph[node]
        for nxt in outs:
            if nxt in path:   # prevent cycles
                continue

            if nxt not in reachable_to_end:
                continue  # pruning: cannot reach end from here

            if nxt in required:
                new_seen = seen_required | {nxt}
            else:
                new_seen = seen_required

            stack.append((nxt, path + (nxt,), new_seen))

    return paths

with open('input.txt','r') as f:
    inputs=[line.strip() for line in f.readlines()]

graph = parse_graph(inputs)
paths = all_paths_with_required_fast(graph, "svr", "out", ['fft','dac'])

print("Total paths:", len(paths))
for p in paths:
    print(" -> ".join(p))

KeyboardInterrupt: 

In [ ]:
#not working part2
def parse_graph(lines):
    graph = {}
    for line in lines:
        if not line.strip():
            continue
        name, targets = line.split(":")
        name = name.strip()
        targets = targets.strip().split()
        graph[name] = targets
    return graph


def count_paths_required(graph, start, end, required_nodes):
    # -----------------------------------------------------
    # Step 1 — Map nodes to integer IDs (fast lookup)
    nodes = list(graph.keys())
    for outs in graph.values():
        for node in outs:
            if node not in graph:
                nodes.append(node)

    nodes = sorted(set(nodes))
    id_map = {name: i for i, name in enumerate(nodes)}
    n = len(nodes)

    start_id = id_map[start]
    end_id = id_map[end]

    # Build adjacency list of IDs
    adj = [[] for _ in range(n)]
    for src, outs in graph.items():
        s = id_map[src]
        for o in outs:
            adj[s].append(id_map[o])

    # -----------------------------------------------------
    # Step 2 — Precompute reachability to "end"
    rev = [[] for _ in range(n)]
    for s in range(n):
        for o in adj[s]:
            rev[o].append(s)

    reachable = [False] * n
    reachable[end_id] = True
    stack = [end_id]

    while stack:
        cur = stack.pop()
        for prev in rev[cur]:
            if not reachable[prev]:
                reachable[prev] = True
                stack.append(prev)

    # -----------------------------------------------------
    # Step 3 — Build required bitmask
    required_nodes = [r for r in required_nodes if r in id_map]
    req_bits = {id_map[name]: (1 << i) for i, name in enumerate(required_nodes)}
    required_full_mask = (1 << len(required_nodes)) - 1

    # -----------------------------------------------------
    # Step 4 — DFS + Memoization (DP)
    # visited_mask tracks visited nodes to avoid cycles
    from functools import lru_cache

    @lru_cache(maxsize=None)
    def dfs(node_id, req_mask, visited_mask):
        # prune: cannot reach end from here
        if not reachable[node_id]:
            return 0

        # reached end: check if all required nodes were hit
        if node_id == end_id:
            return 1 if req_mask == required_full_mask else 0

        total = 0
        for nxt in adj[node_id]:

            bit = 1 << nxt
            if visited_mask & bit:
                continue  # cycle prevention

            new_req_mask = req_mask
            if nxt in req_bits:
                new_req_mask |= req_bits[nxt]

            total += dfs(nxt, new_req_mask, visited_mask | bit)

        return total

    # initial state
    start_req_mask = req_bits[start_id] if start_id in req_bits else 0
    start_visited = 1 << start_id

    return dfs(start_id, start_req_mask, start_visited)

with open("input.txt") as f:
    inputs = [line.strip() for line in f]
graph = parse_graph(inputs)
count = count_paths_required(graph, "svr", "out", ["fft", "dac"])
print("Total matching paths:", count)


In [ ]:
#part2 working
#SCC Condensation → DAG → Bitmask DP for Required-Node Path Counting
#determinstic
from collections import defaultdict, deque

def parse_graph(lines):
    graph = {}
    for line in lines:
        if not line.strip():
            continue
        name, targets = line.split(":")
        name = name.strip()
        targets = targets.strip().split()
        graph[name] = targets
    return graph

# ---------------------------------------------------------
# Kosaraju SCC (Strongly Connected Components)
def compute_scc(graph):
    # Ensure all nodes appear in graph by adding missing keys
    missing = set()
    for outs in graph.values():
        for v in outs:
            if v not in graph:
                missing.add(v)

    # Now safely add missing nodes
    for v in missing:
        graph[v] = []

    nodes = list(graph.keys())

    # -------------------------
    # First DFS (postorder)
    visited = set()
    order = []

    def dfs1(u):
        visited.add(u)
        for v in graph[u]:
            if v not in visited:
                dfs1(v)
        order.append(u)

    for node in nodes:
        if node not in visited:
            dfs1(node)

    # -------------------------
    # Reverse graph
    rev = defaultdict(list)
    for u in graph:
        for v in graph[u]:
            rev[v].append(u)

    # -------------------------
    # Second DFS (component assign)
    comp_id = {}
    current_comp = 0

    def dfs2(u):
        comp_id[u] = current_comp
        for v in rev[u]:
            if v not in comp_id:
                dfs2(v)

    for u in reversed(order):
        if u not in comp_id:
            dfs2(u)
            current_comp += 1

    return comp_id, current_comp

# ---------------------------------------------------------
# Build condensed DAG
def build_condensed_dag(graph, comp_id, num_comps):
    dag = [set() for _ in range(num_comps)]
    comp_nodes = [[] for _ in range(num_comps)]

    # Group original nodes into SCC components
    for node, cid in comp_id.items():
        comp_nodes[cid].append(node)

    # Add DAG edges
    for u in graph:
        cu = comp_id[u]
        for v in graph[u]:
            cv = comp_id[v]
            if cu != cv:
                dag[cu].add(cv)

    # Convert sets to lists for speed
    dag = [list(neigh) for neigh in dag]
    return dag, comp_nodes

# ---------------------------------------------------------
# DP over condensed DAG (bitmask for required nodes)
def count_paths_scc_dp(graph, start, end, required_nodes):

    # ---- Build SCCs ----
    comp_id, num_comps = compute_scc(graph)

    # Condensed DAG
    dag, comp_nodes = build_condensed_dag(graph, comp_id, num_comps)

    start_c = comp_id[start]
    end_c = comp_id[end]

    # -----------------------------------------------------
    # Compute reachability to end
    rev = [[] for _ in range(num_comps)]
    for u in range(num_comps):
        for v in dag[u]:
            rev[v].append(u)

    reachable = [False] * num_comps
    reachable[end_c] = True
    queue = [end_c]

    while queue:
        x = queue.pop()
        for y in rev[x]:
            if not reachable[y]:
                reachable[y] = True
                queue.append(y)

    # -----------------------------------------------------
    # Map required nodes to required bits by component
    required_nodes = [r for r in required_nodes if r in comp_id]
    req_bits = {}
    for i, r in enumerate(required_nodes):
        req_bits[r] = (1 << i)

    full_mask = (1 << len(required_nodes)) - 1

    # Each component may contain multiple required nodes
    comp_required_mask = [0] * num_comps
    for node, cid in comp_id.items():
        if node in req_bits:
            comp_required_mask[cid] |= req_bits[node]

    # -----------------------------------------------------
    # Topological ordering since DAG
    indeg = [0]*num_comps
    for u in range(num_comps):
        for v in dag[u]:
            indeg[v] += 1

    topo = []
    q = deque()

    for i in range(num_comps):
        if indeg[i] == 0:
            q.append(i)

    while q:
        x = q.popleft()
        topo.append(x)
        for y in dag[x]:
            indeg[y] -= 1
            if indeg[y] == 0:
                q.append(y)

    dp = [defaultdict(int) for _ in range(num_comps)]
    # Start mask
    start_mask = comp_required_mask[start_c]
    dp[start_c][start_mask] = 1

    for u in topo:
        if not reachable[u]:
            continue  # prune unreachable components
        for mask, count in list(dp[u].items()):
            for v in dag[u]:
                if not reachable[v]:
                    continue

                new_mask = mask | comp_required_mask[v]
                dp[v][new_mask] += count

    return dp[end_c][full_mask]


with open("input.txt") as f:
    inputs = [line.strip() for line in f]

graph = parse_graph(inputs)
count = count_paths_scc_dp(graph, "svr", "out", ["fft", "dac"])

print(count)


490695961032000
